In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import scipy.ndimage as ndimage
from skimage.morphology import ball, binary_closing, remove_small_objects, binary_dilation, binary_erosion

c:\Users\DELL\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [3]:
path_to_data = "../results/femur_tibia_mask.nii.gz"

In [4]:
mask_vol = nib.load(path_to_data)
mask_vol


Summary of Segmentation Mask
Shape: (512, 512, 216) → 216 axial slices of 512×512 pixels

Voxel size: (0.869 × 0.869 × 2.0 mm³) 

Datatype: 8-bit unsigned integer (uint8) — suitable for labeled masks

Physical size of volume (approx.):

Width: 512 × 0.869 mm ≈ 445 mm

Height: 512 × 0.869 mm ≈ 445 mm

Depth: 216 × 2.0 mm = 432 mm


In [5]:
mask = mask_vol.get_fdata() # get image array data
affine = mask_vol.affine # gets affine transformation matrix associated with the volume data
affine


array([[-8.69140983e-01,  0.00000000e+00,  0.00000000e+00,
         2.22500000e+02],
       [ 0.00000000e+00, -8.69140983e-01,  0.00000000e+00,
         2.22500000e+02],
       [ 0.00000000e+00,  0.00000000e+00,  2.00000000e+00,
        -9.00500000e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])


Matrix that describes the relationship between voxel coordinates in the image data array and real-world coordinates in a reference space

